In [3]:
import os
import requests
import polars as pl
import pandas as pd
from bs4 import BeautifulSoup
races = pd.read_csv("data_raw/ergast/races.csv")

In [4]:
pl.Config(tbl_rows=50)

In [5]:
races = pl.from_pandas(races)
races

raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,2009,1,1,"""Australian Grand Prix""","""2009-03-29""","""06:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
2,2009,2,2,"""Malaysian Grand Prix""","""2009-04-05""","""09:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
3,2009,3,17,"""Chinese Grand Prix""","""2009-04-19""","""07:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
4,2009,4,3,"""Bahrain Grand Prix""","""2009-04-26""","""12:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
5,2009,5,4,"""Spanish Grand Prix""","""2009-05-10""","""12:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
6,2009,6,6,"""Monaco Grand Prix""","""2009-05-24""","""12:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
7,2009,7,5,"""Turkish Grand Prix""","""2009-06-07""","""12:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
8,2009,8,9,"""British Grand Prix""","""2009-06-21""","""12:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
9,2009,9,20,"""German Grand Prix""","""2009-07-12""","""12:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""


In [6]:
races.with_columns(pl.col('url').map_elements(lambda x: x.split('/')[-1] + ".html").alias('filename')).select(pl.col(['raceId','filename']))

C:\Users\micha\AppData\Local\Temp\ipykernel_26156\2474267249.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  races.with_columns(pl.col('url').map_elements(lambda x: x.split('/')[-1] + ".html").alias('filename')).select(pl.col(['raceId','filename']))


raceId,filename
i64,str
1,"""2009_Australian_Grand_Prix.htm…"
2,"""2009_Malaysian_Grand_Prix.html"""
3,"""2009_Chinese_Grand_Prix.html"""
4,"""2009_Bahrain_Grand_Prix.html"""
5,"""2009_Spanish_Grand_Prix.html"""
6,"""2009_Monaco_Grand_Prix.html"""
7,"""2009_Turkish_Grand_Prix.html"""
8,"""2009_British_Grand_Prix.html"""
9,"""2009_German_Grand_Prix.html"""


In [7]:
dowl_loc = "data_raw/downloads/wikipedia/races"

In [8]:
def scrape_gp(source):
    with open(os.path.join(dowl_loc, source), 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file.read())
        gp = {"filename": source}
        data = ["Course length","Course","Distance","Scheduled distance","Weather","Date","Official name","Attendance"]

        if "safety car" in soup.text.lower().replace("virtual safety car","xxx"):
            gp['Safety_car'] = True
        if "virtual safety car" in soup.text.lower():
            gp['Virtual_safety_car'] = True
        
        tables = soup.find_all(class_="infobox")
        for table in tables:
            if "Race details" in table.text:
                rows = table.find_all("tr")
        
        for r in rows:
            for d in data:
                try:
                    if r.find("th").text.strip() == d:
                        gp[d] = r.find('td').text.strip()
                    if (r.find("th").text.strip() == 'Time'):
                        if 'Pole_position' not in gp.keys():
                            gp['Pole_position'] = r.find('td').text.strip()
                        else:
                            gp['Fastest_lap'] = r.find('td').text.strip()
                        
                except:
                    pass            
            
    return gp

In [9]:
%%timeit
scrape_gp("1988_German_Grand_Prix.html")

219 ms ± 12.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
scrape_gp("1988_German_Grand_Prix.html")

{'filename': '1988_German_Grand_Prix.html',
 'Date': '24 July 1988',
 'Official name': 'Mobil 1 Grosser Preis von Deutschland',
 'Course': 'Permanent racing facility',
 'Course length': '6.802 km (4.227 miles)',
 'Distance': '44 laps, 299.068 km (185.832 miles)',
 'Weather': 'Wet and cool',
 'Pole_position': '1:44.596',
 'Fastest_lap': '2:03.032 on lap 40'}

In [11]:
gps = []
counter = 0
for file in os.listdir("data_raw/downloads/wikipedia/races"):
    counter += 1
    if counter % 50 == 0:
        print(f"{counter}: {file}")
    gps.append(scrape_gp(file))

50: 1956_Belgian_Grand_Prix.html
100: 1961_Italian_Grand_Prix.html
150: 1966_United_States_Grand_Prix.html
200: 1971_Canadian_Grand_Prix.html
250: 1974_United_States_Grand_Prix.html
300: 1978_Belgian_Grand_Prix.html
350: 1981_Dutch_Grand_Prix.html
400: 1984_Italian_Grand_Prix.html
450: 1987_Portuguese_Grand_Prix.html
500: 1990_United_States_Grand_Prix.html
550: 1994_Belgian_Grand_Prix.html
600: 1997_Austrian_Grand_Prix.html
650: 2000_Brazilian_Grand_Prix.html
700: 2003_Brazilian_Grand_Prix.html
750: 2005_United_States_Grand_Prix.html
800: 2008_Monaco_Grand_Prix.html
850: 2011_Indian_Grand_Prix.html
900: 2014_Austrian_Grand_Prix.html
950: 2016_Malaysian_Grand_Prix.html
1000: 2019_Austrian_Grand_Prix.html
1050: 2021_Russian_Grand_Prix.html
1100: 2023_United_States_Grand_Prix.html


In [12]:
gps

[{'filename': '1950_Belgian_Grand_Prix.html',
  'Date': '18 June 1950',
  'Official name': 'Grand Prix Automobile de Belgique',
  'Course': 'Permanent racing circuit',
  'Course length': '14.120 km (8.825 miles)',
  'Distance': '35 laps, 494.2 km (308.875 miles)',
  'Weather': 'Warm, dry and sunny',
  'Pole_position': '4:37.0',
  'Fastest_lap': '4:34.1 on lap 18[1]'},
 {'filename': '1950_British_Grand_Prix.html',
  'Date': '13 May 1950',
  'Official name': "The Royal Automobile Club Grand Prix d'Europe Incorporating The British Grand Prix",
  'Course': 'Permanent racing facility',
  'Course length': '4.649\xa0km (2.889\xa0mi)',
  'Distance': '70 laps, 325.430\xa0km (202.213\xa0mi)',
  'Weather': 'Sunny, mild, dry.',
  'Attendance': '100,000[1]',
  'Pole_position': '1:50.8[2]',
  'Fastest_lap': '1:50.6[3] on lap 2'},
 {'filename': '1950_French_Grand_Prix.html',
  'Date': '2 July 1950',
  'Official name': "XXXVII Grand Prix de l'A.C.F.",
  'Course': 'Temporary road course',
  'Course len

In [13]:
pl.DataFrame(gps)

filename,Date,Official name,Course,Course length,Distance,Weather,Pole_position,Fastest_lap,Attendance,Scheduled distance
str,str,str,str,str,str,str,str,str,str,str
"""1950_Belgian_Grand_Prix.html""","""18 June 1950""","""Grand Prix Automobile de Belgi…","""Permanent racing circuit""","""14.120 km (8.825 miles)""","""35 laps, 494.2 km (308.875 mil…","""Warm, dry and sunny""","""4:37.0""","""4:34.1 on lap 18[1]""",null,null
"""1950_British_Grand_Prix.html""","""13 May 1950""","""The Royal Automobile Club Gran…","""Permanent racing facility""","""4.649 km (2.889 mi)""","""70 laps, 325.430 km (202.213 m…","""Sunny, mild, dry.""","""1:50.8[2]""","""1:50.6[3] on lap 2""","""100,000[1]""",null
"""1950_French_Grand_Prix.html""","""2 July 1950""","""XXXVII Grand Prix de l'A.C.F.""","""Temporary road course""","""7.815 km (4.856 miles)""","""64 laps, 500.160 km (310.785 m…","""Hot and sunny""","""2:30.6""","""2:35.6 on lap 7[1]""",null,null
"""1950_Indianapolis_500.html""","""May 30, 1950 (1950-5-30)""",null,"""Permanent racing facility""","""4.023 km (2.500 miles)""","""138 laps, 555.224 km (345.000 …",null,null,null,null,"""200 laps, 804.672 km (500.000 …"
"""1950_Italian_Grand_Prix.html""","""3 September 1950""","""XXI GRAN PREMIO D'ITALIA""","""Permanent racing facility""","""6.300 km (3.915 miles)""","""80 laps, 504.000 km (313.171 m…","""Warm, hot and sunny""","""1:58.6""","""2:00.0 on lap 7[3]""",null,null
"""1950_Monaco_Grand_Prix.html""","""21 May 1950""","""Prix de Monte-Carlo et XIe Gra…","""Street circuit""","""3.180 km (1.976 miles)""","""100 laps, 318.000 km (197.596 …",null,"""1:50.2[1]""","""1:51.0[2]""",null,null
"""1950_Swiss_Grand_Prix.html""","""4 June 1950""","""Großer Preis der Schweiz für A…","""Temporary street/road circuit""","""7.28 km (4.524 miles)""","""42 laps, 305.760 km (190.008 m…","""Warm, dry and sunny""","""2:42.1""","""2:41.6 on lap 8[1]""",null,null
"""1951_Belgian_Grand_Prix.html""","""17 June 1951""","""XIII Grote Prijs Van Belgie""","""Grand Prix Circuit""","""14.12 km (8.774 miles)""","""36 laps, 508.320 km (315.864 m…","""Sunny, Mild, Dry""","""4:25.0""","""4:22.1 on lap 10""",null,null
"""1951_British_Grand_Prix.html""","""14 July 1951""","""IV RAC British Grand Prix""","""Permanent racing facility""","""4.649 km (2.888 miles)""","""90 laps, 408.410 km (253.774 m…","""Mild, Dry""","""1:43.4""","""1:44.0 on lap 38""",null,null


In [14]:
pl.DataFrame(gps).write_json("data_raw/wiki_gps_raw.json")

In [15]:
pl.DataFrame(gps).filter(pl.col("Distance").is_null())

filename,Date,Official name,Course,Course length,Distance,Weather,Pole_position,Fastest_lap,Attendance,Scheduled distance
str,str,str,str,str,str,str,str,str,str,str
"""1974_Austrian_Grand_Prix.html""","""18 August 1974""","""XII Memphis Grand Prix von Öst…","""Permanent racing facility""","""5.911 km (3.673 miles)""",null,null,"""1:35.40[1]""","""1:37.22[2] on lap 46""",null,null
"""1974_Belgian_Grand_Prix.html""","""12 May 1974""",null,"""Permanent racing facility""","""3.72 km (2.314 miles)""",null,null,"""1:09.82[1]""","""1:11.31[2] on lap 37""",null,null
"""1974_Dutch_Grand_Prix.html""","""23 June 1974""","""XXI Grand Prix Zandvoort""","""Permanent racing facility""","""4.226 km (2.626[1] miles)""",null,null,"""1:18.31[2]""","""1:21.44[3] on lap 63""",null,null


In [16]:
pl.DataFrame(gps).filter(pl.col("Course").is_null())

filename,Date,Official name,Course,Course length,Distance,Weather,Pole_position,Fastest_lap,Attendance,Scheduled distance
str,str,str,str,str,str,str,str,str,str,str
"""1975_Belgian_Grand_Prix.html""","""May 25, 1975""",null,null,"""4.262 km (2.648 miles)""","""70 laps, 298.340 km (185.380 m…","""Dry and sunny""","""1:25.43[1]""","""1:26.76[2] on lap 11""",null,null
"""1975_Swedish_Grand_Prix.html""","""8 June 1975""",null,null,"""4.018[1] km (2.497 miles)""","""80 laps, 321.440[1] km (199.73…","""Dry""","""1:24.630[2]""","""1:28.267[3] on lap 61""",null,null
"""1976_Belgian_Grand_Prix.html""","""May 16, 1976""","""XXXIV Grote Prijs van Belgie[1…",null,"""4.262 km (2.648 miles)""","""70 laps, 298.34 km (185.38 mil…",null,"""1:26.55[2]""","""1:25.98[3]""",null,null
"""1976_Swedish_Grand_Prix.html""","""13 June 1976""","""VII Gislaved Sveriges Grand Pr…",null,"""4.018[2] km (2.497 miles)""","""72 laps, 289.296 km (179.760 m…","""Dry""","""1:25.659[3]""","""1:28.002 on lap 11[4]""",null,null
"""1977_Belgian_Grand_Prix.html""","""5 June 1977""",null,null,"""4.262 km (2.648 miles)""","""70 laps, 298.340 km (185.380 m…","""Wet""","""1:24.64""","""1:27.36 on lap 53""",null,null
"""1977_French_Grand_Prix.html""","""3 July 1977""",null,null,"""3.801 km (2.361 miles)""","""80 laps, 304.08 km (188.88 mil…","""Dry""","""1:12.21""","""1:13.75 on lap 76""",null,null
"""1978_Belgian_Grand_Prix.html""","""21 May 1978""",null,null,"""4.262 km (2.648 miles)""","""70 laps, 298.340 km (185.380 m…","""Dry""","""1:20.90""","""1:23.13 on lap 66""",null,null
"""1979_Belgian_Grand_Prix.html""","""13 May 1979""",null,null,"""4.262 km (2.648 miles)""","""70 laps, 298.340 km (185.380 m…","""Dry""","""1:21.13""","""1:23.09 on lap 63""",null,null
"""1979_French_Grand_Prix.html""","""July 1, 1979""",null,null,"""3.801 km (2.361 miles)""","""80 laps, 304.08 km (188.88 mil…","""Dry""","""1:07.19""","""1:09.16 on lap 71""",null,null


In [18]:
pl.DataFrame(gps).select(pl.col('Course')).unique()

Course
str
"""Semi-permanent racing facility…"
"""Permanent road course"""
"""Road-based with permanent infr…"
"""Semi-permanent racing facility"""
"""Korea International Circuit"""
null
"""Permanent Racing Facility"""
"""Street circuit/Racing facility"""
"""Permanent race track"""


In [20]:
pl.DataFrame(gps).filter(pl.col("Course").str.contains("treet"))

filename,Date,Official name,Course,Course length,Distance,Weather,Pole_position,Fastest_lap,Attendance,Scheduled distance
str,str,str,str,str,str,str,str,str,str,str
"""1950_Monaco_Grand_Prix.html""","""21 May 1950""","""Prix de Monte-Carlo et XIe Gra…","""Street circuit""","""3.180 km (1.976 miles)""","""100 laps, 318.000 km (197.596 …",null,"""1:50.2[1]""","""1:51.0[2]""",null,null
"""1950_Swiss_Grand_Prix.html""","""4 June 1950""","""Großer Preis der Schweiz für A…","""Temporary street/road circuit""","""7.28 km (4.524 miles)""","""42 laps, 305.760 km (190.008 m…","""Warm, dry and sunny""","""2:42.1""","""2:41.6 on lap 8[1]""",null,null
"""1951_Spanish_Grand_Prix.html""","""28 October 1951""","""XI Gran Premio de España""","""Street circuit""","""6.316 km (3.925 miles)""","""70 laps, 442.120 km (274.721 m…","""Hot, Dry""","""2:10.59""","""2:16.93""",null,null
"""1951_Swiss_Grand_Prix.html""","""27 May 1951""","""XI Großer Preis der Schweiz""","""Temporary street/road circuit""","""7.28 km (4.524 miles)""","""42 laps, 305.760 km (190.008 m…","""Overcast, rain""","""2:35.9""","""2:51.1""",null,null
"""1952_Swiss_Grand_Prix.html""","""18 May 1952""","""XII Großer Preis der Schweiz""","""Temporary street/road circuit""","""7.280 km (4.524 miles)""","""62 laps, 451.360 km (280.462 m…","""Sunny""","""2:47.5""","""2:49.1 on lap 46""",null,null
"""1953_Swiss_Grand_Prix.html""","""23 August 1953""","""XIII Großer Preis der Schweiz""","""Temporary street/road circuit""","""7.280 km (4.524 miles)""","""65 laps, 473.200 km (294.033 m…","""Sunny, mild, dry""","""2:40:1""","""2:41.3 on lap 50""",null,null
"""1954_Spanish_Grand_Prix.html""","""24 October 1954""","""XII Gran Premio de España""","""Temporary street circuit""","""6.316 km (3.925 miles)""","""80 laps, 505.280 km (313.966 m…","""Sunny, mild, dry""","""2:18.1""","""2:20.4 on lap 3""",null,null
"""1954_Swiss_Grand_Prix.html""","""22 August 1954""","""XIV Großer Preis der Schweiz""","""Temporary street/road circuit""","""7.280 km (4.524 miles)""","""66 laps, 480.480 km (298.556 m…","""Rain""","""2:39.5""","""2:39.7 on lap 34""",null,null
"""1955_Monaco_Grand_Prix.html""","""22 May 1955""","""XIII Grand Prix Automobile de …","""Street circuit""","""3.145 km (1.955 miles)""","""100 laps, 314.5 km (195.5 mile…",null,"""1:41.1""","""1:42.4""",null,null


In [22]:
pl.DataFrame(gps).filter(pl.col("Course") == "Temporary road circuit")

filename,Date,Official name,Course,Course length,Distance,Weather,Pole_position,Fastest_lap,Attendance,Scheduled distance
str,str,str,str,str,str,str,str,str,str,str
"""1962_Belgian_Grand_Prix.html""","""17 June 1962""","""XXII Grote Prijs van Belgie""","""Temporary road circuit""","""14.100 km (8.761 miles)""","""32 laps, 451.200 km (280.363 m…","""Sunny""","""3:57.0""","""3:55.6 on lap 15""",null,null
"""1962_French_Grand_Prix.html""","""8 July 1962""","""XLVIII Grand Prix de l'A.C.F.""","""Temporary road circuit""","""6.542 km (4.065 miles)""","""54 laps, 353.268 km (219.511 m…","""Sunny""","""2:14.8""","""2:16.9 on lap 32""",null,null
"""1962_South_African_Grand_Prix.…","""29 December 1962""","""9th International RAC Grand Pr…","""Temporary road circuit""","""3.920 km (2.436 miles)""","""82 laps, 321.415 km (199.718 m…","""Dry, windy""","""1:29.3""","""1:31.0 on lap 3""",null,null
"""1963_South_African_Grand_Prix.…","""28 December 1963""","""10th International RAC Grand P…","""Temporary road circuit""","""3.920 km (2.436 miles)""","""85 laps, 333.175 km (207.025 m…","""Hot, dry and windy""","""1:28.9""","""1:29.1 on lap 33""",null,null
"""1965_South_African_Grand_Prix.…","""1 January 1965""","""11th International RAC Grand P…","""Temporary road circuit""","""3.920 km (2.436 miles)""","""85 laps, 333.175 km (207.025 m…",null,"""1:27.2""","""1:27.6 on lap 80""",null,null


In [24]:
pl.DataFrame(gps).filter(pl.col("Weather").is_null())

filename,Date,Official name,Course,Course length,Distance,Weather,Pole_position,Fastest_lap,Attendance,Scheduled distance
str,str,str,str,str,str,str,str,str,str,str
"""1950_Indianapolis_500.html""","""May 30, 1950 (1950-5-30)""",null,"""Permanent racing facility""","""4.023 km (2.500 miles)""","""138 laps, 555.224 km (345.000 …",null,null,null,null,"""200 laps, 804.672 km (500.000 …"
"""1950_Monaco_Grand_Prix.html""","""21 May 1950""","""Prix de Monte-Carlo et XIe Gra…","""Street circuit""","""3.180 km (1.976 miles)""","""100 laps, 318.000 km (197.596 …",null,"""1:50.2[1]""","""1:51.0[2]""",null,null
"""1951_Indianapolis_500.html""","""May 30, 1951 (1951-5-30)""",null,"""Permanent racing facility""","""4.023 km (2.500 miles)""","""200 laps, 804.672 km (500.000 …",null,null,null,null,null
"""1952_Indianapolis_500.html""","""May 30, 1952 (1952-5-30)""",null,"""Permanent racing facility""","""4.023 km (2.500 miles)""","""200 laps, 804.672 km (500.000 …",null,null,null,null,null
"""1953_Indianapolis_500.html""","""May 30, 1953 (1953-5-30)""",null,"""Permanent racing facility""","""4.023 km (2.500 miles)""","""200 laps, 804.672 km (500.000 …",null,null,null,null,null
"""1954_Indianapolis_500.html""","""May 31, 1954 (1954-5-31)""",null,"""Permanent racing facility""","""4.023 km (2.500 miles)""","""200 laps, 804.672 km (500.000 …",null,null,null,null,null
"""1955_Belgian_Grand_Prix.html""","""June 5, 1955""","""XVII Grote Prijs Van Belgie""","""Permanent racing facility""","""14.120 km (8.774 miles)""","""36 laps, 508.320 km (315.855 m…",null,"""4:18.1""","""4:20.6""",null,null
"""1955_Indianapolis_500.html""","""May 30, 1955 (1955-5-30)""",null,"""Permanent racing facility""","""4.023 km (2.500 miles)""","""200 laps, 804.672 km (500.000 …",null,null,null,null,null
"""1955_Italian_Grand_Prix.html""","""September 11, 1955""","""XXVI Gran Premio d'Italia""","""Permanent road course""","""10.000 km (6.214 miles)""","""50 laps, 500.023 km (310.700 m…",null,"""2:46.5""","""2:46.9""",null,null


In [26]:
pl.DataFrame(gps).filter(pl.col("Attendance").is_null())

filename,Date,Official name,Course,Course length,Distance,Weather,Pole_position,Fastest_lap,Attendance,Scheduled distance
str,str,str,str,str,str,str,str,str,str,str
"""1950_Belgian_Grand_Prix.html""","""18 June 1950""","""Grand Prix Automobile de Belgi…","""Permanent racing circuit""","""14.120 km (8.825 miles)""","""35 laps, 494.2 km (308.875 mil…","""Warm, dry and sunny""","""4:37.0""","""4:34.1 on lap 18[1]""",null,null
"""1950_French_Grand_Prix.html""","""2 July 1950""","""XXXVII Grand Prix de l'A.C.F.""","""Temporary road course""","""7.815 km (4.856 miles)""","""64 laps, 500.160 km (310.785 m…","""Hot and sunny""","""2:30.6""","""2:35.6 on lap 7[1]""",null,null
"""1950_Indianapolis_500.html""","""May 30, 1950 (1950-5-30)""",null,"""Permanent racing facility""","""4.023 km (2.500 miles)""","""138 laps, 555.224 km (345.000 …",null,null,null,null,"""200 laps, 804.672 km (500.000 …"
"""1950_Italian_Grand_Prix.html""","""3 September 1950""","""XXI GRAN PREMIO D'ITALIA""","""Permanent racing facility""","""6.300 km (3.915 miles)""","""80 laps, 504.000 km (313.171 m…","""Warm, hot and sunny""","""1:58.6""","""2:00.0 on lap 7[3]""",null,null
"""1950_Monaco_Grand_Prix.html""","""21 May 1950""","""Prix de Monte-Carlo et XIe Gra…","""Street circuit""","""3.180 km (1.976 miles)""","""100 laps, 318.000 km (197.596 …",null,"""1:50.2[1]""","""1:51.0[2]""",null,null
"""1950_Swiss_Grand_Prix.html""","""4 June 1950""","""Großer Preis der Schweiz für A…","""Temporary street/road circuit""","""7.28 km (4.524 miles)""","""42 laps, 305.760 km (190.008 m…","""Warm, dry and sunny""","""2:42.1""","""2:41.6 on lap 8[1]""",null,null
"""1951_Belgian_Grand_Prix.html""","""17 June 1951""","""XIII Grote Prijs Van Belgie""","""Grand Prix Circuit""","""14.12 km (8.774 miles)""","""36 laps, 508.320 km (315.864 m…","""Sunny, Mild, Dry""","""4:25.0""","""4:22.1 on lap 10""",null,null
"""1951_British_Grand_Prix.html""","""14 July 1951""","""IV RAC British Grand Prix""","""Permanent racing facility""","""4.649 km (2.888 miles)""","""90 laps, 408.410 km (253.774 m…","""Mild, Dry""","""1:43.4""","""1:44.0 on lap 38""",null,null
"""1951_French_Grand_Prix.html""","""1 July 1951""","""XXXVIII Grand Prix de l'ACF""","""Reims-Gueux""","""7.816 km (4.856 miles)""","""77 laps, 601.832 km (373.961 m…","""Sunny, Hot, Dry""","""2:25.7""","""2:27.8 on lap 32""",null,null


In [28]:
def get_time(fastest_lap):
    time = fastest_lap.split(" ")[0].split("[")[0].strip()
    if ":" in time:
        min = time.split(":")[0]
        sec = time.split(":")[1]
    else:
        min = time.split(".")[0]
        sec = ".".join(time.split(".")[1:])
    return (int(min) * 60) + float(sec)

In [30]:
get_time("9:44.6")

584.6

In [32]:
def dist(description):
    laps = float(description.split("laps")[0].strip())
    distance = float(description.split("laps, ")[1].split('km')[0].split('[')[0].strip())
    return {'Laps':laps, 'Kms':distance}

In [70]:
pl.from_pandas(pd.DataFrame(gps))

filename,Date,Official name,Course,Course length,Distance,Weather,Pole_position,Fastest_lap,Attendance,Scheduled distance,Safety_car,Virtual_safety_car
str,str,str,str,str,str,str,str,str,str,str,bool,bool
"""1950_Belgian_Grand_Prix.html""","""18 June 1950""","""Grand Prix Automobile de Belgi…","""Permanent racing circuit""","""14.120 km (8.825 miles)""","""35 laps, 494.2 km (308.875 mil…","""Warm, dry and sunny""","""4:37.0""","""4:34.1 on lap 18[1]""",null,null,null,null
"""1950_British_Grand_Prix.html""","""13 May 1950""","""The Royal Automobile Club Gran…","""Permanent racing facility""","""4.649 km (2.889 mi)""","""70 laps, 325.430 km (202.213 m…","""Sunny, mild, dry.""","""1:50.8[2]""","""1:50.6[3] on lap 2""","""100,000[1]""",null,null,null
"""1950_French_Grand_Prix.html""","""2 July 1950""","""XXXVII Grand Prix de l'A.C.F.""","""Temporary road course""","""7.815 km (4.856 miles)""","""64 laps, 500.160 km (310.785 m…","""Hot and sunny""","""2:30.6""","""2:35.6 on lap 7[1]""",null,null,null,null
"""1950_Indianapolis_500.html""","""May 30, 1950 (1950-5-30)""",null,"""Permanent racing facility""","""4.023 km (2.500 miles)""","""138 laps, 555.224 km (345.000 …",null,null,null,null,"""200 laps, 804.672 km (500.000 …",null,null
"""1950_Italian_Grand_Prix.html""","""3 September 1950""","""XXI GRAN PREMIO D'ITALIA""","""Permanent racing facility""","""6.300 km (3.915 miles)""","""80 laps, 504.000 km (313.171 m…","""Warm, hot and sunny""","""1:58.6""","""2:00.0 on lap 7[3]""",null,null,null,null
"""1950_Monaco_Grand_Prix.html""","""21 May 1950""","""Prix de Monte-Carlo et XIe Gra…","""Street circuit""","""3.180 km (1.976 miles)""","""100 laps, 318.000 km (197.596 …",null,"""1:50.2[1]""","""1:51.0[2]""",null,null,null,null
"""1950_Swiss_Grand_Prix.html""","""4 June 1950""","""Großer Preis der Schweiz für A…","""Temporary street/road circuit""","""7.28 km (4.524 miles)""","""42 laps, 305.760 km (190.008 m…","""Warm, dry and sunny""","""2:42.1""","""2:41.6 on lap 8[1]""",null,null,null,null
"""1951_Belgian_Grand_Prix.html""","""17 June 1951""","""XIII Grote Prijs Van Belgie""","""Grand Prix Circuit""","""14.12 km (8.774 miles)""","""36 laps, 508.320 km (315.864 m…","""Sunny, Mild, Dry""","""4:25.0""","""4:22.1 on lap 10""",null,null,null,null
"""1951_British_Grand_Prix.html""","""14 July 1951""","""IV RAC British Grand Prix""","""Permanent racing facility""","""4.649 km (2.888 miles)""","""90 laps, 408.410 km (253.774 m…","""Mild, Dry""","""1:43.4""","""1:44.0 on lap 38""",null,null,null,null


In [62]:
races

raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,2009,1,1,"""Australian Grand Prix""","""2009-03-29""","""06:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
2,2009,2,2,"""Malaysian Grand Prix""","""2009-04-05""","""09:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
3,2009,3,17,"""Chinese Grand Prix""","""2009-04-19""","""07:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
4,2009,4,3,"""Bahrain Grand Prix""","""2009-04-26""","""12:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
5,2009,5,4,"""Spanish Grand Prix""","""2009-05-10""","""12:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
6,2009,6,6,"""Monaco Grand Prix""","""2009-05-24""","""12:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
7,2009,7,5,"""Turkish Grand Prix""","""2009-06-07""","""12:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
8,2009,8,9,"""British Grand Prix""","""2009-06-21""","""12:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
9,2009,9,20,"""German Grand Prix""","""2009-07-12""","""12:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""


In [72]:
df = races.with_columns(
    pl.col('url').map_elements(lambda x: x.split('/')[-1] + ".html").alias('filename')
).select(pl.col(['raceId','filename'])).join(
    pl.from_pandas(pd.DataFrame(gps)).with_columns(
        pl.col("Distance").map_elements(dist).alias('x')
    ).unnest('x'), how='left', on='filename'
).with_columns(
    pl.col("Fastest_lap").map_elements(get_time)
).select(
    pl.col(
        ['raceId','Date','Official name','Distance',"Scheduled distance",'Course','Weather',
         'Attendance','Laps','Kms','Fastest_lap','Safety_car','Virtual_safety_car']
    )
).with_columns(
    pl.col('raceId').cast(float)
)

df

C:\Users\micha\AppData\Local\Temp\ipykernel_26156\2120507315.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = races.with_columns(
C:\Users\micha\AppData\Local\Temp\ipykernel_26156\2120507315.py:4: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  pl.from_pandas(pd.DataFrame(gps)).with_columns(
C:\Users\micha\AppData\Local\Temp\ipykernel_26156\2120507315.py:7: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  ).with_columns(


raceId,Date,Official name,Distance,Scheduled distance,Course,Weather,Attendance,Laps,Kms,Fastest_lap,Safety_car,Virtual_safety_car
f64,str,str,str,str,str,str,str,f64,f64,f64,bool,bool
1.0,"""29 March 2009""","""2009 Formula 1 ING Australian …","""58 laps, 307.574 km (191.118 m…",null,"""Temporary street circuit""","""Sunny with temperatures reachi…",null,58.0,307.574,87.706,true,null
2.0,"""5 April 2009""","""2009 Formula 1 Petronas Malays…","""31 laps, 171.833 km (106.772 m…","""56 laps, 310.408 km (192.879 m…","""Permanent racing facility""","""Dry start, with heavy rain and…","""97,368 (Weekend)[2]""",31.0,171.833,96.641,true,null
3.0,"""19 April 2009""","""2009 Formula 1 Chinese Grand P…","""56 laps, 305.066 km (189.559 m…",null,"""Permanent racing facility""","""Rain""","""120,000 (Weekend)[2]""",56.0,305.066,112.592,true,null
4.0,"""26 April 2009""","""2009 Formula 1 Gulf Air Bahrai…","""57 laps, 308.238 km (191.530 m…",null,"""Permanent racing facility""","""Sunny""","""93,680 (Weekend)[2]""",57.0,308.238,94.556,null,null
5.0,"""10 May 2009""","""Formula 1 Gran Premio de Españ…","""66 laps, 307.104 km (190.826 m…",null,"""Permanent racing facility""","""Warm, Sunny""","""92,500""",66.0,307.104,82.762,true,null
6.0,"""24 May 2009""","""Formula 1 Grand Prix de Monaco…","""78 laps, 260.52 km (162.24 mil…",null,"""Street circuit""","""Sunny""",null,78.0,260.52,75.154,null,null
7.0,"""7 June 2009""","""2009 Formula 1 ING Turkish Gra…","""58 laps, 309.396 km (192.250 m…",null,"""Permanent racing facility""","""Sunny""","""36,000 (Weekend)[2]""",58.0,309.396,87.579,null,null
8.0,"""21 June 2009""","""2009 Formula 1 Santander Briti…","""60 laps, 308.355 km (191.603 m…",null,"""Permanent racing facility""","""Dry, overcast""","""128,000""",60.0,308.355,80.735,null,null
9.0,"""12 July 2009""","""Formula 1 Grosser Preis Santan…","""60 laps, 308.863 km (191.919 m…",null,"""Permanent racing facility""","""Sunny and overcast""",null,60.0,308.863,93.365,null,null


In [46]:
df.filter(pl.col("Fastest_lap").is_null())

raceId,Date,Official name,Distance,Scheduled distance,Course,Weather,Attendance,Laps,Kms,Fastest_lap
f64,str,str,str,str,str,str,str,f64,f64,f64
748.0,"""May 30, 1960 (1960-5-30)""",null,"""200 laps, 804.672 km (500.000 …",null,"""Permanent racing facility""",null,null,200.0,804.672,null
757.0,"""May 30, 1959 (1959-5-30)""",null,"""200 laps, 804.672 km (500.000 …",null,"""Permanent racing facility""",null,null,200.0,804.672,null
768.0,"""May 30, 1958 (1958-5-30)""",null,"""200 laps, 804.672 km (500.000 …",null,"""Permanent racing facility""",null,null,200.0,804.672,null
778.0,"""May 30, 1953 (1953-5-30)""",null,"""200 laps, 804.672 km (500.000 …",null,"""Permanent racing facility""",null,null,200.0,804.672,null
786.0,"""May 30, 1956 (1956-5-30)""",null,"""200 laps, 804.672 km (500.000 …",null,"""Permanent racing facility""",null,null,200.0,804.672,null
794.0,"""May 30, 1955 (1955-5-30)""",null,"""200 laps, 804.672 km (500.000 …",null,"""Permanent racing facility""",null,null,200.0,804.672,null
800.0,"""May 31, 1954 (1954-5-31)""",null,"""200 laps, 804.672 km (500.000 …",null,"""Permanent racing facility""",null,null,200.0,804.672,null
809.0,"""May 30, 1953 (1953-5-30)""",null,"""200 laps, 804.672 km (500.000 …",null,"""Permanent racing facility""",null,null,200.0,804.672,null
818.0,"""May 30, 1952 (1952-5-30)""",null,"""200 laps, 804.672 km (500.000 …",null,"""Permanent racing facility""",null,null,200.0,804.672,null


In [38]:
df.filter(pl.col("Date").str.contains("1977"))

raceId,Date,Official name,Distance,Scheduled distance,Course,Weather,Attendance,Laps,Kms,Fastest_lap
f64,str,str,str,str,str,str,str,f64,f64,f64
543.0,"""9 January 1977""",null,"""53 laps, 307.93 km (191.33 mil…",null,"""Permanent road course""","""Dry""",null,53.0,307.93,111.06
544.0,"""23 January 1977""",null,"""40 laps, 318.400[1] km (197.84…",null,"""Permanent race track""","""Dry""",null,40.0,318.4,154.55
545.0,"""5 March 1977""","""XXIII The Citizen Grand Prix o…","""78 laps, 320.112 km (198.908 m…",null,"""Permanent racing facility""","""Sunny""",null,78.0,320.112,77.63
546.0,"""April 3, 1977""","""Long Beach Grand Prix""","""80 laps, 260.08 km (161.60 mil…",null,"""Temporary street course""","""Temperatures of 68 °F (20 °C);…",null,80.0,260.08,82.753
547.0,"""8 May 1977""","""XXIII Gran Premio de España[1]""","""75 laps, 255.3 km (158.63 mile…",null,"""Permanent racing facility""",null,null,75.0,255.3,80.81
548.0,"""22 May 1977""",null,"""76 laps, 251.712 km (156.406 m…",null,"""Street circuit""","""Dry""",null,76.0,251.712,91.07
549.0,"""5 June 1977""",null,"""70 laps, 298.340 km (185.380 m…",null,null,"""Wet""",null,70.0,298.34,87.36
550.0,"""19 June 1977""",null,"""72 laps, 289.296 km (179.760 m…",null,"""Permanent racing facility""","""Dry""",null,72.0,289.296,87.607
551.0,"""3 July 1977""",null,"""80 laps, 304.08 km (188.88 mil…",null,null,"""Dry""",null,80.0,304.08,73.75


In [40]:
df.filter(pl.col('Course').is_null())

raceId,Date,Official name,Distance,Scheduled distance,Course,Weather,Attendance,Laps,Kms,Fastest_lap
f64,str,str,str,str,str,str,str,f64,f64,f64
480.0,"""29 August 1982""",null,"""80 laps, 304.08 km (188.88 mil…",null,null,"""Dry""",null,80.0,304.08,67.477
487.0,"""17 May 1981""",null,"""54 laps, 230.148 km (143.007 m…","""70 laps, 298.340 km (185.380 m…",null,"""Dry, then wet at the end""",null,54.0,230.148,83.3
502.0,"""May 4, 1980""",null,"""72 laps, 306.864 km (190.066 m…",null,null,"""Dry""",null,72.0,306.864,80.88
517.0,"""13 May 1979""",null,"""70 laps, 298.340 km (185.380 m…",null,null,"""Dry""",null,70.0,298.34,83.09
519.0,"""July 1, 1979""",null,"""80 laps, 304.08 km (188.88 mil…",null,null,"""Dry""",null,80.0,304.08,69.16
532.0,"""21 May 1978""",null,"""70 laps, 298.340 km (185.380 m…",null,null,"""Dry""",null,70.0,298.34,83.13
549.0,"""5 June 1977""",null,"""70 laps, 298.340 km (185.380 m…",null,null,"""Wet""",null,70.0,298.34,87.36
551.0,"""3 July 1977""",null,"""80 laps, 304.08 km (188.88 mil…",null,null,"""Dry""",null,80.0,304.08,73.75
564.0,"""May 16, 1976""","""XXXIV Grote Prijs van Belgie[1…","""70 laps, 298.34 km (185.38 mil…",null,null,null,null,70.0,298.34,85.98


In [74]:
df.write_parquet('data/gps_wikipedia.parquet')